# Assignment 1

In this assignment, you will implement a multi-layer perceptron (MLP) using PyTorch.


## Task 1

In [ ]:
def f(x):
    return 3 * x ** 2 - 4 * x + 5

def gradient(f, x, h=0.0001):
    return (f(x + h) - f(x)) / h

gradient(f, 2)

In [ ]:
# What happens if we keep decreasing h?
gradient(f, 2, h=0.0000001)

# Why is the gradient 0?
# If you don't know, google it!
# ------------------
# Write your implementation here.
# ANSWER:
# ------------------

## Task 2

MNIST

## Extra Credit

ㅊㅚ소값 한번에